In [1]:
# from emnist import extract_training_samples,extract_test_samples
# train_images, train_labels = extract_training_samples('byclass')
# test_images, test_labels = extract_test_samples('byclass')

In [1]:
import tensorflow as tf

In [2]:
def mish(x):
    return x * tf.math.tanh(tf.math.softplus(x))

In [3]:
class Conv2D(tf.keras.layers.Conv2D):
    def __init__(self,f,k,a,b,**kwdarg):
        super().__init__(
            filters=f,
            kernel_size=k,
            activation=a,
            use_bias=b,
            strides=1,
            padding='valid',
            **kwdarg)
    def call(self,inp,training):
        x = super().call(inp)
        return x

In [4]:
class DepthwiseConv2D(tf.keras.layers.DepthwiseConv2D):
    def __init__(self,m,k,a,b,**kwdarg):
        super().__init__(
            depth_multiplier=m,
            kernel_size=k,
            activation=a,
            use_bias=b,
            strides=1,
            padding='valid',
            **kwdarg)
    def call(self,inp,training):
        x = super().call(inp)
        return x

In [5]:
class DepthControl(tf.keras.layers.Layer):
    def __init__(self,r,a,b,groups=1):
        super().__init__()
        self.r = 2**r
        self.a = a
        self.b = b
        self.groups = groups
    def build(self,input_shape):
        self.conv = Conv2D(
            f=int(input_shape[-1]*self.r),
            k=1,
            a=self.a,
            b=self.b,
            groups=self.groups
        )
    def call(self,inp,training):
        x = self.conv(inp,training=training)
        return x

In [6]:
class SubBlock(tf.keras.layers.Layer):
    def __init__(self,vision_layer,nins):
        super().__init__()
        self.vision_layer = vision_layer
        self.main = tf.keras.Sequential([DepthControl(r=r,a=mish,b=True) for r in nins]+
                                        [DepthControl(r=-r,a=mish,b=True) for r in nins[::-1]])
        self.mix = DepthControl(r=-1,a=mish,b=False)
    def call(self,inp,training):
        x1 = self.vision_layer(inp,training=training)
        x2 = self.main(x1,training=training)
        x = tf.concat([x1,x2],axis=-1)
        x = self.mix(x,training=training)
        return x

In [7]:
class GroupSubBlock(tf.keras.layers.Layer):
    def __init__(self,vision_layer,groups,nins):
        super().__init__()
        self.vision_layer = vision_layer
        self.main = tf.keras.Sequential([DepthControl(r=r,a=mish,b=True,groups=groups) for r in nins]+
                                        [DepthControl(r=-r,a=mish,b=True,groups=groups) for r in nins[::-1]])
    def call(self,inp,training):
        x1 = self.vision_layer(inp,training=training)
        x2 = self.main(x1,training=training)
        x = x1 + x2
        return x

In [8]:
class Contract(tf.keras.layers.Layer):
    def __init__(self):
        super().__init__()
        self.downsample = tf.keras.layers.AveragePooling2D(pool_size=2, strides=2, padding='valid')
        self.increase = DepthControl(r=1,a=None,b=False)
    def call(self,inp,training):
        x = self.downsample(inp,training=training)
        x = self.increase(x,training=training)
        return x

In [9]:
class Model(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.block1 = tf.keras.Sequential([
            SubBlock(DepthwiseConv2D(m=8,k=3,a=None,b=False),[1,1]),
            SubBlock(Conv2D(f=8,k=3,a=mish,b=True),[1,1]),
            Contract()])
        self.block2 = tf.keras.Sequential([
            SubBlock(Conv2D(f=16,k=3,a=mish,b=True),[1,0,0]),
            Contract()])
        self.block3 = tf.keras.Sequential([
            SubBlock(Conv2D(f=32,k=3,a=mish,b=True),[1,0,0]),
            SubBlock(Conv2D(f=32,k=3,a=mish,b=True),[1,0,0]),
            Contract()])
        self.block4 = tf.keras.Sequential([
            SubBlock(Conv2D(f=64,k=2,a=mish,b=True),[-1,0,0]),
            tf.keras.layers.Lambda(lambda x: tf.tile(x,[1,1,1,12]))
        ])
        self.block5 = tf.keras.Sequential([
            GroupSubBlock(Conv2D(f=1032,k=(1,2),a=mish,b=True,groups=12),12,[0,0]), #25
            GroupSubBlock(Conv2D(f=1032,k=(1,2),a=mish,b=True,groups=12),12,[0,0]), #24
            GroupSubBlock(Conv2D(f=1032,k=(1,2),a=mish,b=True,groups=12),12,[0,0]), #23
            tf.keras.layers.AveragePooling2D(pool_size=(1,3), strides=(1,2), padding='valid'), #11
            GroupSubBlock(Conv2D(f=2064,k=(1,2),a=mish,b=True,groups=12),12,[0,0]), #10
            GroupSubBlock(Conv2D(f=2064,k=(1,2),a=mish,b=True,groups=12),12,[0,0]), #9
            tf.keras.layers.AveragePooling2D(pool_size=(1,3), strides=(1,2), padding='valid'), #4
            GroupSubBlock(Conv2D(f=4128,k=(1,2),a=mish,b=True,groups=12),12,[0,0]), #3
            GroupSubBlock(Conv2D(f=4128,k=(1,2),a=mish,b=True,groups=12),12,[0,0]), #2
            GroupSubBlock(Conv2D(f=4128,k=(1,2),a=mish,b=True,groups=12),12,[0,0]), #1
            
            tf.keras.layers.Lambda(lambda x: tf.squeeze(x,axis=[1,2])),
            tf.keras.layers.Reshape([1,12,-1])
        ])
        self.output_layer = tf.keras.Sequential([
            Conv2D(f=1024,k=1,a=mish,b=True),
            Conv2D(f=512,k=1,a=mish,b=True),
            Conv2D(f=256,k=1,a=mish,b=True),
            Conv2D(f=128,k=1,a=mish,b=True),
            Conv2D(f=64,k=1,a=mish,b=True),
            Conv2D(f=36,k=1,a=None,b=False),
            tf.keras.layers.Lambda(lambda x: tf.squeeze(x,axis=1))
        ])
        
    def call(self,image,training):
        x = self.block1(image,training=training)
        x = self.block2(x,training=training)
        x = self.block3(x,training=training)
        x = self.block4(x,training=training)
        x = self.block5(x,training=training)
        x = self.output_layer(x,training=training)
        return x
    
    def predict(self,image):
        raw_predict = self(image,training=False)
        predict = tf.argmax(raw_predict,axis=-1)
        predict = predict.numpy()
        predict = [utils.token2char(p) for p in predict]
        return predict

In [10]:
model = Model()

In [11]:
import FLAGS

FLAGS.MODEL = model
FLAGS.CORPUS = ' 0123456789ABCDEFGHIJKLMNPQRSTUVWXYZ'
FLAGS.IMAGE_SHAPE = (40,240)
FLAGS.SEQUENCE_SIZE = 12

FLAGS.TRAIN.RECORD_WEIGHTS_PATH = r'D:\DL_competition_Dataset\ChinaSteel\OCR_model_weights\model_6\trial_1'
# FLAGS.TRAIN.RECORD_WEIGHTS_PATH = r'C:\Users\Atlas\Desktop\weights'
FLAGS.TRAIN.LABEL_SMOOTHING = 0.01
FLAGS.TRAIN.OPTIMIZER.TYPE = tf.keras.optimizers.Adam
FLAGS.TRAIN.OPTIMIZER.MAX_LR = 0.5 * 1e-3
FLAGS.TRAIN.OPTIMIZER.SCHEDULE_GAMMA = -2
FLAGS.TRAIN.EPOCH.TOTAL = 300
FLAGS.TRAIN.EPOCH.WARMUP = 2

FLAGS.DATA.TRAIN.TABLE_PATH = 'cleaned_merge_table.csv'
# FLAGS.DATA.TRAIN.IMAGE_PATH = r'C:\Users\Atlas\Desktop\ChinaSteel_dataset\custom\train_segment'
FLAGS.DATA.TRAIN.IMAGE_PATH = r'D:\DL_competition_Dataset\ChinaSteel\dataset\custom\segment\rotated_train'
FLAGS.DATA.TRAIN.TRAIN_BATCH_SIZE = 6
FLAGS.DATA.TRAIN.VALIDATION_RATIO = 0.05
FLAGS.DATA.TRAIN.VALIDATION_BATCH_SIZE = 8
# FLAGS.DATA.TEST.IMAGE_PATH = r'C:\Users\Atlas\Desktop\ChinaSteel_dataset\custom\test_segment'
FLAGS.DATA.TEST.IMAGE_PATH = r'D:\DL_competition_Dataset\ChinaSteel\dataset\custom\segment\rotated_test'
FLAGS.DATA.TEST.BATCH_SIZE = 8

import utils

In [12]:
# model.load_weights(r'D:\DL_competition_Dataset\ChinaSteel\OCR_model_weights\model_6\trial_1\0053_1p55522\weights')
# table = utils.recognize(model,r'D:\DL_competition_Dataset\ChinaSteel\dataset\private\rotated')
# table.to_csv(r'C:\Users\user\Dropbox\DL_Competitions\ChinaSteel\submission\private_submission_1.csv',index=False,encoding='utf8')
# table.head()

,id,text
0,115WbJDuFNePtmmPPbfXCiS8xbEma,1L5914032A
1,1167tlVRmziSI87CbfMU42Im6Mps5a,2G0763012A
2,11E6V_Rkh9hp6xs9bRVsI21h2ZQUBf,2FR802013F
3,12nOyH6Mqf3xsVh2pvoXJKdNFiHqZB5,1WL106032Y
4,12OfZfvTIhBltWQtFzU37bIZXbx77ea,1WG072012H


In [12]:
model.load_weights(r'D:\DL_competition_Dataset\ChinaSteel\OCR_model_weights\model_6\trial_1_finetune\0002_0p70234_SOTA\weights')
table = utils.recognize(model,r'D:\DL_competition_Dataset\ChinaSteel\dataset\private\rotated')
table.to_csv(r'C:\Users\user\Dropbox\DL_Competitions\ChinaSteel\submission\private_submission_2.csv',index=False,encoding='utf8')
table.head()

,id,text
0,115WbJDuFNePtmmPPbfXCiS8xbEma,1L5914032A
1,1167tlVRmziSI87CbfMU42Im6Mps5a,2G0763012A
2,11E6V_Rkh9hp6xs9bRVsI21h2ZQUBf,2FR802013F
3,12nOyH6Mqf3xsVh2pvoXJKdNFiHqZB5,1WL106032Y
4,12OfZfvTIhBltWQtFzU37bIZXbx77ea,1WG072012H
